In [ ]:
#VPN --> Connect to Server 
#Remote Desktop --> Sypder --> type to change to M: drive
#Type in jupyter notebook in terminal 
#Open same notebook

In [ ]:
import time, os
import pandas as pd
import multiprocessing
import numpy as np

In [ ]:
#Finding outcomes of interest from D_ITEMS (dictionary)

d_items = pd.read_csv('data-raw/mimic/D_ITEMS.csv')


In [ ]:
#Create function to find dialysis events 

#dialysis ITEMIDs: https://github.com/MIT-LCP/mimic-code/blob/master/concepts/rrt.sql
dialysis_items=d_items[d_items.CATEGORY=='Dialysis'].ITEMID.unique().tolist()
print(len(dialysis_items))
d_items.LABEL=d_items.LABEL.map(lambda x: str(x).lower())
def find_item(string): 
    items=['dialy', 'dialysis', 'tunneled', 'crrt', 'cvvh', 'scuf']
    for i in items: 
        if i in string: return True 
d_items['Dialysis']=d_items.LABEL.map(lambda x: 1 if find_item(x) else 0)
for i in d_items[d_items.Dialysis==1].ITEMID.unique().tolist(): 
    if i not in dialysis_items: dialysis_items.append(i)
print(len(dialysis_items))

In [ ]:
#Check
30044 in d_items.ITEMID.unique()

In [ ]:
#Create function to find pressor events 

#pressors ITEMIDs 
#https://github.com/MIT-LCP/mimic-code/blob/master/concepts/durations/vasopressin_dose.sql

pressors_items=[30044, 30119, 30309, 30051, 30047, 30120,  30043, 30307, 30127, 30128]
print(len(pressors_items))
d_items.LABEL=d_items.LABEL.map(lambda x: str(x).lower())
def find_item(string): 
    items=['neo ', 'levophed', 'vasopressin', 'epinephrin']
    for i in items: 
        if i in string: return True 
d_items['Pressors']=d_items.LABEL.map(lambda x: 1 if find_item(x) else 0)
for i in d_items[d_items.Pressors==1].ITEMID.unique().tolist(): 
    if i not in pressors_items: pressors_items.append(i)
print(len(pressors_items))
not_wanted=[6016, 8339]
pressors_items=[i for i in pressors_items if i not in not_wanted]
print(len(pressors_items))

In [ ]:
#Create function to find mechanical ventilation events 

#MV: https://github.com/MIT-LCP/mimic-code/blob/master/concepts/durations/ventilation_classification.sql
MV_items=[720, 223849, 223848, 445, 448, 449, 450, 1340, 1486, 1600, 224687, 
          639, 654, 681, 682, 683, 684,224685,224684,224686, 
          218,436,535,444,224697,224695,224696,224746,224747,
          221,1,1211,1655,2000,226873,224738,224419,224750,227187, 
          543,  5865,5866,224707,224709,224705,224706,
          60,437,505,506,686,220339,224700, 3459, 
          501,502,503,224702, 223,667,668,669,670,671,672, 224701]
print(len(MV_items))
d_items.LABEL=d_items.LABEL.map(lambda x: str(x).lower())
def find_item(string): 
    items=['ett']
    for i in items: 
        if i in string: return True 
d_items['ETT']=d_items.LABEL.map(lambda x: 1 if find_item(x) else 0)
for i in d_items[d_items.ETT==1].ITEMID.unique().tolist(): 
    if i not in MV_items: 
        MV_items.append(i)
print(len(MV_items))
not_wanted=[45943, 80020, 80276, 225754, 224755, 224837, 224838, 227980, 220120, 223867]
MV_items=[i for i in MV_items if i not in not_wanted]
print(len(MV_items))

In [ ]:
#Create function to find trach events 

#Trach 
trach_items=[]
d_items.LABEL=d_items.LABEL.map(lambda x: str(x).lower())
def find_item(string): 
    items=['trach']
    for i in items: 
        if i in string: return True 
d_items['Trach']=d_items.LABEL.map(lambda x: 1 if find_item(x) else 0)
for i in d_items[d_items.Trach==1].ITEMID.unique().tolist(): 
    if i not in trach_items: trach_items.append(i)
print(len(trach_items))
not_wanted=[80147, 4588]
trach_items=[i for i in trach_items if i not in not_wanted]
print(len(trach_items))

In [ ]:
#Function to find outcomes of interests 
def find_item_id(integer, item_list): 
    if integer in item_list: return 1
    else: return 0

In [ ]:
%%time # 20 seconds.

# Read InputEvents.CSV, which will find pressor events 

input_df=pd.read_csv('data-raw/mimic/INPUTEVENTS_CV.csv')
#df=pd.read_csv('CHARTEVENTS.csv', usecols=['SUBJECT_ID', 'HADM_ID', 'ITEMID'], assume_missing=True)

print(input_df.shape)
input_df.head(5)

In [ ]:
input_df['Pressors']=input_df.ITEMID.map(lambda x: find_item_id(int(x), pressors_items))

In [ ]:
input_final=input_df[input_df.Pressors==1]
print(input_final.shape)

In [ ]:
input_final.to_csv('data/MIMIC_001_inputevents.csv')

In [ ]:
%%time
# Read ChartEvents.CSV (33 GB), which will have all other outcome events 

cache_file = "data/cache/chart-events.feather"
if not os.path.exists(cache_file):
    # This takes X seconds to load.
    df = pd.read_csv('data-raw/mimic/CHARTEVENTS.csv')
    df.to_feather(cache_file)
else:
    # Takes Y seconds to load - nice speedup.
    df = pd.read_feather(cache_file)

#df=pd.read_csv('CHARTEVENTS.csv', usecols=['SUBJECT_ID', 'HADM_ID', 'ITEMID'], assume_missing=True)

df.head(5)

In [ ]:
df['Mech_Vent']=df.ITEMID.map(lambda x: find_item_id(int(x), MV_items))

In [ ]:
df['Dialysis']=df.ITEMID.map(lambda x: find_item_id(int(x), dialysis_items))

In [ ]:
df['Trach']=df.ITEMID.map(lambda x: find_item_id(int(x), trach_items))

In [ ]:
df_final=df[(df.Mech_Vent==1) | (df.Dialysis==1) |(df.Trach==1)]

In [ ]:
df_final.shape

In [ ]:
df_final.to_csv('data/MIMIC_001_chartevents.csv')

In [ ]:
df_final.head(5)